In [ ]:
!pip install -q transformers
!pip install -q evaluate

# <font color='orange'> Import library </font>

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import time

import torch
from torch import nn
import evaluate

# <font color='orange'> Create Pytorch Dataset </font>

In [ ]:
import numpy as np
from PIL import Image
from torch.utils.data import Dataset

class ImageSegmentationDataset(Dataset):
  """Image segmentation dataset."""

  def __init__(self, dataset, data_path, transform):
    """
    Args:
        dataset
    """
    self.dataset = dataset
    self.data_path = data_path
    self.transform = transform

  def __len__(self):
    return len(self.dataset)

  def __getitem__(self, idx):
    img_path = os.path.join(self.data_path, self.dataset.iloc[idx, 0])
    label_path = os.path.join(self.data_path, self.dataset.iloc[idx, 1])
    image_label = self.dataset.iloc[idx, 2]

    original_image = Image.open(img_path).convert('RGB')
    original_segmentation_map = Image.open(label_path)

    original_image = np.array(original_image)
    original_segmentation_map = np.array(original_segmentation_map.convert('1'))*1

    transformed = self.transform(image=original_image, mask=original_segmentation_map)
    image, segmentation_map = transformed['image'], transformed['mask']

    # convert to C, H, W
    image = image.transpose(2, 0, 1)

    return image, segmentation_map, original_image, original_segmentation_map, image_label

In [ ]:
def train_val_split(dataset, validation_split=0.2):
    """make dataloader for pytorch training

    Args:
        dataset ([object]): [the dataset object]
        train_batch_size ([int]): [training batch size]
        validation_split (float, optional): [validation ratio]. Defaults to 0.2.

    Returns:
        [type]: [description]
    """
    # number of samples in train and test set
    train_len = int(len(dataset) * (1 - validation_split))
    test_len = len(dataset) - train_len
    train_set, test_set = torch.utils.data.random_split(dataset, [train_len, test_len])

    return train_set, test_set

In [ ]:
import albumentations as A

ADE_MEAN = np.array([123.675, 116.280, 103.530]) / 255
ADE_STD = np.array([58.395, 57.120, 57.375]) / 255

train_transform = A.Compose([
    A.RandomCrop(width=224, height=224),
    A.HorizontalFlip(p=0.5),
    A.Normalize(mean=ADE_MEAN, std=ADE_STD),
])

test_transform = A.Compose([
    A.Resize(width=224, height=224),
    A.Normalize(mean=ADE_MEAN, std=ADE_STD),

])

In [ ]:
def compute_samples_weight(train_data, train_set):
    y_train_indices = train_set.indices
    y_train = [train_data.label[i] for i in y_train_indices]
    class_sample_count = np.array([len(np.where(y_train == t)[0]) for t in np.unique(y_train)])

    weight = 1. / class_sample_count
    samples_weight = np.array([weight[t] for t in y_train])
    samples_weight = torch.from_numpy(samples_weight)
    return samples_weight

In [ ]:
from sklearn import preprocessing

img_labels = ['adenocarcinoma','high-grade in','low-grade in','normal','polyp','serrated adenoma']
le = preprocessing.LabelEncoder()
le.fit(img_labels)

datasets = []
sample_weights = []

for i in range(0,5):
    df_train = pd.read_csv(f'/kaggle/input/ebhi-seg-csv/all/train_fold_{i}.csv')
    df_test = pd.read_csv(f'/kaggle/input/ebhi-seg-csv/all/test_fold_{i}.csv')
    
    df_train.label = le.transform(df_train.label)
    df_test.label = le.transform(df_test.label)

    ## train test split
    dataset = ImageSegmentationDataset(df_train, data_path='/kaggle/input/ebhi-seg/', transform=train_transform)
    train_dataset, valid_dataset = train_val_split(dataset, validation_split=0.5)
    test_dataset = ImageSegmentationDataset(df_test, data_path='/kaggle/input/ebhi-seg/', transform=test_transform)
    datasets.append((train_dataset, valid_dataset, test_dataset))
    
    #compute random sample weight
    samples_weight = compute_samples_weight(df_train, train_dataset)
    sample_weights.append(samples_weight)

# <font color='orange'> Create PyTorch DataLoaders </font>

In [ ]:
from transformers import Mask2FormerImageProcessor

# Create a preprocessor
preprocessor = Mask2FormerImageProcessor(
    ignore_index=-1, reduce_labels=False, do_resize=False,
    do_rescale=False, do_normalize=False,
    num_classes=2,
)

In [ ]:
from torch.utils.data import DataLoader

def collate_fn(batch):
    inputs = list(zip(*batch))
    images = inputs[0]
    segmentation_maps = inputs[1]
    # this function pads the inputs to the same size,
    # and creates a pixel mask
    # actually padding isn't required here since we are cropping
    batch = preprocessor(
        images,
        segmentation_maps=segmentation_maps,
        return_tensors="pt",
    )

    batch["original_images"] = inputs[2]
    batch["original_segmentation_maps"] = inputs[3]
    batch['imgage_label'] = inputs[4]
    
    return batch

In [ ]:
dataloaders = []
for i, (train_set, valid_set, test_set) in enumerate(datasets):
    samples_weight = sample_weights[i]
    sampler = torch.utils.data.sampler.WeightedRandomSampler(
        samples_weight.type('torch.DoubleTensor'), len(samples_weight))
    
    train_dataloader = DataLoader(train_set, batch_size=8, shuffle=True, collate_fn=collate_fn)
    val_dataloader = DataLoader(valid_set, batch_size=4, shuffle=True, collate_fn=collate_fn)
    test_dataloader = DataLoader(test_set, batch_size=4, shuffle=False, collate_fn=collate_fn)
    
    dataloaders.append((train_dataloader, val_dataloader, test_dataloader))

# <font color='orange'> Verify data (!!) </font>

In [ ]:
import torch

batch = next(iter(train_dataloader))
for k,v in batch.items():
    print(type(v))
    if isinstance(v, torch.Tensor):
        print(k,v.shape)
    else:
        if isinstance(v[0], str):
            print(k, ':', v[0])
        else:
            print(k,v[0].shape)

# <font color='orange'>  Define model </font>

In [ ]:
import copy
from transformers import Mask2FormerForUniversalSegmentation
from torch import Tensor

class SegmentationNetwork(nn.Module):
    def __init__(self):
        super(SegmentationNetwork, self).__init__()
        
        # Tạo instance của Mask2FormerForSegmentation
        self.mask2former = Mask2FormerForUniversalSegmentation.from_pretrained(
            "facebook/mask2former-swin-base-coco-panoptic")

    def forward(self, pixel_values, class_labels=None, mask_labels=None):
        outputs = self.mask2former(
            pixel_values=pixel_values, 
            class_labels=class_labels, 
            mask_labels=mask_labels
        )
        return outputs

In [ ]:
import torch.nn.functional as F

class ClassificationNetwork(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        
        self.n_classes = output_dim
        
        self.input_fc = nn.Linear(input_dim, 250)
        self.hidden_fc = nn.Linear(250, 100)
        self.output_fc = nn.Linear(100, output_dim)
        
    def forward(self, x):
        #x = [batch size, height, width]
        batch_size = x.shape[0]
        x = x.view(batch_size, -1)
        
        #x = [batch size, height * width]
        h_1 = F.relu(self.input_fc(x))
        
        #h_1 = [batch size, 250]
        h_2 = F.relu(self.hidden_fc(h_1))

        #h_2 = [batch size, 100]
        y_pred = self.output_fc(h_2)
        #y_pred = [batch size, output dim]
        
        return y_pred

In [ ]:
class PosthocHybridCBM(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.residual_classifier = nn.Linear(input_dim, output_dim)
    
    def forward(self, x):
        batch_size = x.shape[0]
        x = x.view(batch_size, -1)
        return self.residual_classifier(x)

In [ ]:
class MultiTaskNetwork(nn.Module):
    def __init__(self, seg_model_path=None, clf_model_path=None):
        super(MultiTaskNetwork, self).__init__()
        
        self.seg_loss=None
        self.clf_loss=None
        
        #Load mask2former model
        self.seg_model = SegmentationNetwork()
        if isinstance(seg_model_path, str) and seg_model_path.endswith("pt"):
            #self.seg_model = copy.deepcopy(self.seg_model)
            self.seg_model.load_state_dict(torch.load(seg_model_path))
        
        #Load classification model 
        self.clf_model = ClassificationNetwork(input_dim=1024, output_dim=6)
        if isinstance(clf_model_path, str) and clf_model_path.endswith("pt"):
            self.clf_model.load_state_dict(torch.load(clf_model_path))
            
        #HybridCMB
        #self.residual_clf = PosthocHybridCBM(input_dim=1024, output_dim=6)

    def forward(self, pixel_values, class_labels=None, mask_labels=None):
        seg_outputs = self.seg_model(
            pixel_values=pixel_values, 
            class_labels=class_labels, 
            mask_labels=mask_labels
        )
        
        #outputs = model(pixel_values=pixel_values.to(device))
        fmaps = seg_outputs.encoder_last_hidden_state
        avgPool = nn.AvgPool2d(kernel_size=(7,7))
        
        fmaps = avgPool(fmaps)
        fmaps = fmaps.reshape(fmaps.shape[0], fmaps.shape[1])
        clf_outputs = self.clf_model(fmaps)
        
        return (seg_outputs, clf_outputs)
    
    def predict(self, batch, preprocessor, device):
        seg_outputs = self.seg_model(pixel_values=batch["pixel_values"].to(device))
        #predict segmentation map
        original_images = batch["original_images"]
        target_sizes = [(image.shape[0], image.shape[1]) for image in original_images]
        predicted_segmentation_maps = preprocessor.post_process_semantic_segmentation(
            seg_outputs, target_sizes=target_sizes)

        #get encoder
        fmaps = seg_outputs.encoder_last_hidden_state
        avgPool = nn.AvgPool2d(kernel_size=(7,7))

        #predict class of images
        fmaps = avgPool(fmaps)
        fmaps = fmaps.reshape(fmaps.shape[0], fmaps.shape[1])
        predicted_labels = self.clf_model(fmaps)

        return predicted_segmentation_maps, predicted_labels
            

# <font color='orange'> Training the Model </font>

In [ ]:
from tqdm.auto import tqdm

def train_one_fold(
    model, 
    train_dataloader, 
    val_dataloader,
    id2label,
    device, 
    optimizer,
    #scheduler=None,
    iou_metric,
    #acc_metric,
    idx_fold=0,
    epochs=100
):   
    running_loss = 0.0
    num_samples = 0
    lost_min = np.Inf
    
    #track the step of trainning loss
    seg_losses = []
    cls_losses = []
    mtl_losses = []
    idx_losses = []
    
    for epoch in range(epochs):
        print("Epoch:", epoch)
        start_time = time.time()
        
        model.train()
        for idx, batch in enumerate(tqdm(train_dataloader)):
            target = torch.tensor(batch["imgage_label"])
            target = target.to(device)
            
            # Reset the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            seg_outs, clf_outs = model(
                pixel_values=batch["pixel_values"].to(device),
                class_labels=[labels.to(device) for labels in batch["class_labels"]],
                mask_labels=[labels.to(device) for labels in batch["mask_labels"]],
            )

            # Backward propagation
            seg_loss = seg_outs.loss
            clf_loss = criterion(clf_outs, target)
                
            alpa = 0.5
            loss = alpa*seg_loss + (1-alpa)*clf_loss
            loss.backward()

            running_loss += loss.item()
            
            batch_size = batch["pixel_values"].size(0)
            num_samples += batch_size
            train_loss = running_loss/num_samples
            
            #track train losses
            seg_losses.append((seg_loss.cpu().detach().numpy())[0])
            cls_losses.append(clf_loss.cpu().detach().numpy())
            mtl_losses.append((loss.cpu().detach().numpy())[0])
            idx_losses.append(num_samples)

            if num_samples % 100 == 0:
                print("Loss:", running_loss/num_samples)

            # Optimization
            optimizer.step()
            

        #validate model
        if(train_loss < lost_min):
            lost_min = train_loss
            print(f"Loss = " + str(train_loss) + " ===> Save best epoch")
            torch.save(model.state_dict(), "./multitask_best_fold_" + str(idx_fold) +  ".pt")
        else:
            print("Loss = " + str(train_loss) + " ===> No saving")
            
        end_time = time.time()-start_time
        print(f'Train epoch {epoch} time: {end_time}')
    return lost_min, (idx_losses, seg_losses, cls_losses, mtl_losses)

In [ ]:
def test_result(model, test_dataloader, id2label, device):
    #metric = evaluate.load("mean_iou")
    model.eval()
    
    pred_seg_maps = []
    pred_labels = []
    gnd_truth_seg_maps = []
    gnd_truth_labels = []
    
    for idx, batch in enumerate(tqdm(test_dataloader)): 
        pixel_values = batch["pixel_values"]

        # Forward pass
        with torch.no_grad():
            batch_pred_maps, batch_pred_labels = model.predict(batch, preprocessor, device)
            _, batch_pred_labels = torch.max(batch_pred_labels, 1) 
            batch_pred_labels = batch_pred_labels.cpu().detach().numpy()
        
        pred_seg_maps.extend(batch_pred_maps)
        pred_labels.extend(batch_pred_labels)
        
        # get ground truth segmentation maps
        batch_ground_truth_maps = batch["original_segmentation_maps"]
        gnd_truth_seg_maps.extend(batch_ground_truth_maps)
        metric.add_batch(references=batch_ground_truth_maps, predictions=batch_pred_maps)
        
        #get ground truth label
        batch_gnd_truth_labels = batch["imgage_label"]
        gnd_truth_labels.extend(batch_gnd_truth_labels)
    
    return (gnd_truth_seg_maps, gnd_truth_labels, pred_seg_maps, pred_labels)
    

In [ ]:
##load best model
#test_model = copy.deepcopy(model)
#test_model.load_state_dict(torch.load(f"/kaggle/working/fold0_model.pt"))
#test_model = test_model.to(device)
    
#train_loader, valid_loader, test_loader = dataloaders[0]
#gnd_truth_seg_maps, gnd_truth_labels, pred_seg_maps, pred_labels = test_result(model, test_dataloader, id2label, device)

In [ ]:
import copy

#for idx, (train_loader, valid_loader, test_loader) in enumerate(dataloaders): 
def train_fold_pipeline(idx, model, id2label, device, optimizer, metric, epochs=100):
    print('Training Fold '+ str(idx))
    train_loader, valid_loader, test_loader = dataloaders[idx]
    
    #train the model
    train_iou, train_losses = train_one_fold(model, train_loader, valid_loader, 
          id2label, device, optimizer, metric, idx_fold=idx, epochs=epochs)
    print('train mean IoU: '+ str(train_iou))
    
    #save train load into csv file
    idx_loss, seg_loss, cls_loss, mtl_loss = train_losses
    df_loss = pd.DataFrame(data={
        "batch_idx": idx_loss,
        "seg_loss": seg_loss,
        "cls_loss": cls_loss,
        "mtl_loss": mtl_loss
    })
    df_loss.to_csv("loss.csv", index=None)
    
    #Save check point
    checkpoint = { 
        'epoch': 100,
        'model': model.state_dict(),
        'optimizer':  optimizer.state_dict()}
    torch.save(checkpoint, 'checkpoint.pth')
    
    #load best model
    test_model = copy.deepcopy(model)
    test_model.load_state_dict(torch.load(f"/kaggle/working/multitask_best_fold_{idx}.pt"))
    test_model = test_model.to(device)
    
    #predict segmentation map
    gnd_truth_maps, grd_truth_label, pred_maps, pred_label = test_result(test_model, test_loader, id2label, device)
    metric.add_batch(references=gnd_truth_maps, predictions=pred_maps)
    print("Test mean IoU:", metric.compute(ignore_index=-1, num_labels = len(id2label))['mean_iou'])
    
    return gnd_truth_maps, grd_truth_label, pred_maps, pred_label

# <font color='orange'> Visualize Functions </font>

In [ ]:
def color_palette():
    """Color palette that maps each class to RGB values.

    This one is actually taken from ADE20k.
    """
    return [[0, 0, 0],  [92, 0, 255]]

palette = color_palette()

In [ ]:
def visualize_mask(image: np.ndarray, mask: np.ndarray):
    color_seg = np.zeros((mask.shape[0], mask.shape[1], 3), dtype=np.uint8)
    palette = np.array(color_palette())
    
    for label, color in enumerate(palette):
        color_seg[mask == label, :] = color
    color_seg = color_seg[..., ::-1]  # convert to BGR

    img = np.array(image) * 0.5 + color_seg * 0.5  # plot the image with the segmentation map
    img = img.astype(np.uint8)

    return img

In [ ]:
import os
from PIL import Image
import matplotlib.pyplot as plt

def visualize_seg_mask(test_dataset, predicted_segmentation_maps):
    row_num = 4
    col_num = 5
    step = 5

    fig = plt.figure(figsize=(8, 7))
    for row in range(0, 4):
        idx = np.random.randint(0, len(test_dataset))
        _,_,img, segmentation_map,label = test_dataset[idx]
        
        #img = batch["original_images"][idx]
        #segmentation_map = batch["original_segmentation_maps"][idx]
        pred_segmentation_map = predicted_segmentation_maps[idx].cpu().numpy()

        ax = fig.add_subplot(row_num, col_num, row*step+1, xticks=[], yticks=[])
        ax.imshow(img)
        ax.set_title(label)

        ax = fig.add_subplot(row_num, col_num, row*step+2, xticks=[], yticks=[])
        ax.imshow(segmentation_map)
        ax.set_title('ground truth')

        ax = fig.add_subplot(row_num, col_num, row*step+3, xticks=[], yticks=[])
        ax.imshow(pred_segmentation_map)
        ax.set_title('predict map')

        ax = fig.add_subplot(row_num, col_num, row*step+4, xticks=[], yticks=[])
        grdtruth_mask = visualize_mask(img, segmentation_map)
        ax.imshow(grdtruth_mask)
        ax.set_title('grd truth mask')

        ax = fig.add_subplot(row_num, col_num, row*step+5, xticks=[], yticks=[])
        img_mask = visualize_mask(img, pred_segmentation_map)
        ax.imshow(img_mask)
        ax.set_title('predict mask')

    plt.tight_layout()
    plt.show()

# <font color='orange'> Inference </font>

In [ ]:
import evaluate

id2label = {0:'background',1:'polyp'}
metric = evaluate.load("mean_iou")

## <font color='orange'> With trained weight </font>

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
model = MultiTaskNetwork()
model = model.to(device)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, "min", patience=2, factor=0.5)

In [ ]:
gnd_truth_maps, grd_truth_label, pred_maps, pred_labels = train_fold_pipeline(
    2, model, id2label, device, optimizer, metric, epochs=100)

In [ ]:
#visualize random images
_, _, test_dataset = datasets[3]
visualize_seg_mask(test_dataset, pred_maps)

In [ ]:
#_, _, test_dataset = datasets[0]
visualize_seg_mask(test_dataset, pred_maps)

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
report = classification_report(grd_truth_label, pred_labels, digits=6,target_names=le.classes_)
print(report)

In [ ]:
cm = confusion_matrix(grd_truth_label, pred_labels)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=le.classes_)
disp.plot()

In [ ]:
checkpoint = { 
    'epoch': 100,
    'model': model.state_dict(),
    'optimizer':  optimizer.state_dict(),
    'lr_sched': scheduler}
torch.save(checkpoint, 'checkpoint_final.pth')